In [12]:
import os
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langchain_community.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from openai import OpenAI
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory


In [13]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_ENV = os.getenv('PINECONE_API_ENV')

In [14]:
llm = ChatOpenAI(model="gpt-3.5-turbo")
client = OpenAI(api_key=OPENAI_API_KEY)   
index_name = "arpa"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)


In [20]:
loader = PyPDFLoader("/Users/luisbarajas/Documents/AGI/Papers/imagenet-classification-with-deep-convolutional-neural-networks.pdf")
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)
print(docs)
# res = client.embeddings.create(input=docs, model=MODEL)
# print(res)


[Document(page_content='ImageNet Classiﬁcation with Deep Convolutional\nNeural Networks\nAlex Krizhevsky\nUniversity of Toronto\nkriz@cs.utoronto.caIlya Sutskever\nUniversity of Toronto\nilya@cs.utoronto.caGeoffrey E. Hinton\nUniversity of Toronto\nhinton@cs.utoronto.ca\nAbstract\nWe trained a large, deep convolutional neural network to classify the 1.2 million\nhigh-resolution images in the ImageNet LSVRC-2010 contest into the 1000 dif-\nferent classes. On the test data, we achieved top-1 and top-5 error rates of 37.5%\nand 17.0% which is considerably better than the previous state-of-the-art. The\nneural network, which has 60 million parameters and 650,000 neurons, consists\nof ﬁve convolutional layers, some of which are followed by max-pooling layers,\nand three fully-connected layers with a ﬁnal 1000-way softmax. To make train-\ning faster, we used non-saturating neurons and a very efﬁcient GPU implemen-\ntation of the convolution operation. To reduce overﬁtting in the fully-connec

In [21]:
### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


In [22]:
# Retrieve and generate using the relevant snippets of the blog.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks related to scientific papers, articles and investigations. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [23]:
### Statefully manage chat history ###
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [24]:
conversational_rag_chain.invoke(
    {"input": "give me a resume of the document"},
    config={
        "configurable": {"session_id": "barajas"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'The document evaluates the GPT-4V + Refusal System against a text refusal dataset, achieving a 100% refusal rate. It focuses on testing risks related to the multimodal (vision) functionality of GPT-4, particularly in scientific proficiency, medical advice, stereotyping, disinformation risks, hateful content, and visual vulnerabilities. The evaluations include qualitative and quantitative assessments, such as harmful content refusal evaluations, performance accuracy evaluations for gender, race, and age recognition, privacy assessments, cybersecurity evaluations, and multimodal jailbreak assessments.'